In [1]:
import numpy as np
from src.river import River

env = River()

In [2]:
def value_iter(v=None, mask=None, epsilon=.1, gamma=.9):
    v__ = v if v is not None else np.zeros(env.S.size)
    bell = lambda s,a: sum([env.T(s,a,s_)*(env.R(s,a,s_)+v__[s_]) for s_ in env.S])
    end = False
    while not end:
        v_ = v__.copy()
        for s in env.S:
            if mask is not None and s in mask:
                v__[s] = mask[s] if mask[s] is not None else np.max([bell(s,a) for a in env.A])

        end = np.linalg.norm(v_-v__, ord=np.inf) < (epsilon*(1-gamma))/(2*gamma)
    return v__

     

def lao(gamma=.9):
    v = np.zeros(env.S.size)
    H = np.zeros(env.S.size)

    F = set(env.S0)
    I = set()
    G = set(env.S0)
    GV = set(env.S0)

    end = False

    greedy = lambda s: np.argmax([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])
    transit = lambda s, s_: (env.T(s,greedy(s),s_)>0 and s!=s_)

    def reachable(sG, s=None, S=None, mem=[]):
        # s, S = s or env.S0, S or env.S
        S = S or env.S
        if s is None:
            return any([reachable(sG=sG, s=s_, S=S, mem=mem) for s_ in env.S0])
        if s == sG: return True
        else:
            if s in mem: return False
            else: return any([reachable(sG=sG, s=s_, S=S, mem=mem+[s]) for s_ in S if transit(s,s_)])
    
    getPi = lambda: [greedy(s) for s in env.S]
    while not end:
        s = list( (F&GV)-set(env.G) ).pop()
        F = F-{s}
        I |= {s}
        F |= {x for x in env.S if x not in I and any([env.T(s,a,x)>0 for a in env.A])}
        G = I|F
        Z = {s} | {s_ for s_ in G if reachable(s, s_, G)}
        v = value_iter(v=v, mask={s_:H[s_] if s_ in F else None for s_ in env.S if s_ in Z})
        GV = set([s_ for s_ in G if reachable(s_, S=G)])
        # env.plot(list(v))
        end = len([s for s in env.S if s in list( F&GV ) and s not in env.G]) == 0
    return getPi()
        

pi = lao(gamma=.9)
env.plot(pi, True)

 _____________________________ 
|  →  |  →  |  →  |  →  |  ↓  |
|_____|_____|_____|_____|_____|
|  ↑  |  ↑  |  ↑  |  →  |  ↑  |
|_____|_____|_____|_____|_____|



In [3]:
pi

[0, 0, 0, 2, 0, 2, 2, 2, 2, 1]